In [5]:
import tensorflow as tf

Learning rate를 변경하면서 cost를 비교한다

train/test set으로 나누어 학습시키고 accuracy를 구한다

In [2]:
x_data = [[1, 2, 1],
          [1, 3, 2],
          [1, 3, 4],
          [1, 5, 5],
          [1, 7, 5],
          [1, 2, 5],
          [1, 6, 6],
          [1, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]


# Evaluation our model using this test dataset
x_test = [[2, 1, 1],
          [3, 1, 2],
          [3, 3, 4]]
y_test = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1]]

X = tf.placeholder("float", [None, 3])
Y = tf.placeholder("float", [None, 3])

W = tf.Variable(tf.random_normal([3, 3]))
b = tf.Variable(tf.random_normal([3]))

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
# Try to change learning_rate to small numbers
optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=1e-10).minimize(cost)

# Correct prediction Test model
prediction = tf.arg_max(hypothesis, 1)
is_correct = tf.equal(prediction, tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(201):
        cost_val, W_val, _ = sess.run(
            [cost, W, optimizer], feed_dict={X: x_data, Y: y_data})
        print(step, cost_val, W_val)

    # predict
    print("Prediction:", sess.run(prediction, feed_dict={X: x_test}))
    # Calculate the accuracy
    print("Accuracy: ", sess.run(accuracy, feed_dict={X: x_test, Y: y_test}))

Instructions for updating:
Use `argmax` instead
0 8.727206 [[-0.7115535   0.62098664  0.5261439 ]
 [ 0.4442905  -0.38458702  1.3592359 ]
 [-0.40581986  0.68094885  1.3594123 ]]
1 8.727206 [[-0.7115535   0.62098664  0.5261439 ]
 [ 0.4442905  -0.38458702  1.3592359 ]
 [-0.40581986  0.68094885  1.3594123 ]]
2 8.727206 [[-0.7115535   0.62098664  0.5261439 ]
 [ 0.4442905  -0.38458702  1.3592359 ]
 [-0.40581986  0.68094885  1.3594123 ]]
3 8.727206 [[-0.7115535   0.62098664  0.5261439 ]
 [ 0.4442905  -0.38458702  1.3592359 ]
 [-0.40581986  0.68094885  1.3594123 ]]
4 8.727206 [[-0.7115535   0.62098664  0.5261439 ]
 [ 0.4442905  -0.38458702  1.3592359 ]
 [-0.40581986  0.68094885  1.3594123 ]]
5 8.727206 [[-0.7115535   0.62098664  0.5261439 ]
 [ 0.4442905  -0.38458702  1.3592359 ]
 [-0.40581986  0.68094885  1.3594123 ]]
6 8.727206 [[-0.7115535   0.62098664  0.5261439 ]
 [ 0.4442905  -0.38458702  1.3592359 ]
 [-0.40581986  0.68094885  1.3594123 ]]
7 8.727206 [[-0.7115535   0.62098664  0.5261439 ]

learning rate를 너무 크게주면 cost값이 줄어들지를 않는다

Non-normalized inputs를 학습시키면 결과값이 NaN이 나온다 (값들의 기준이 너무 차이났기 때문)

xy = MinMaxScaler(xy) 라는 함수로 정규화를 시킨 후 학습시키면된다

0~1 사이의 값으로 스케일링한다

In [7]:
import numpy as np

In [10]:
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)

In [11]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

# 가장중요한 부분
# very important. It does not work without it.
xy = MinMaxScaler(xy)
print(xy)

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([4, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(101):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

[[0.99999999 0.99999999 0.         1.         1.        ]
 [0.70548491 0.70439552 1.         0.71881782 0.83755791]
 [0.54412549 0.50274824 0.57608696 0.606468   0.6606331 ]
 [0.33890353 0.31368023 0.10869565 0.45989134 0.43800918]
 [0.51436    0.42582389 0.30434783 0.58504805 0.42624401]
 [0.49556179 0.42582389 0.31521739 0.48131134 0.49276137]
 [0.11436064 0.         0.20652174 0.22007776 0.18597238]
 [0.         0.07747099 0.5326087  0.         0.        ]]
0 Cost:  4.1314335 
Prediction:
 [[-0.6903733]
 [-1.2411041]
 [-1.4032648]
 [-1.5672076]
 [-1.4444995]
 [-1.5791197]
 [-2.01866  ]
 [-2.222594 ]]
1 Cost:  4.1311445 
Prediction:
 [[-0.6902783]
 [-1.2410091]
 [-1.4031849]
 [-1.5671449]
 [-1.4444263]
 [-1.5790486]
 [-2.0186098]
 [-2.2225437]]
2 Cost:  4.1308556 
Prediction:
 [[-0.6901833]
 [-1.2409141]
 [-1.403105 ]
 [-1.5670822]
 [-1.4443529]
 [-1.5789775]
 [-2.0185595]
 [-2.2224934]]
3 Cost:  4.1305666 
Prediction:
 [[-0.6900883]
 [-1.240819 ]
 [-1.4030252]
 [-1.5670195]
 [-1.444